In [1]:
import torch
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torch.utils.data import TensorDataset
from torch.autograd import Variable
from sklearn.metrics import r2_score
from sklearn import preprocessing
#import scipy.io as sio
import numpy as np
import pandas as pd

In [2]:
from scipy.stats import pearsonr

# Parameters

In [3]:
# Define parameters
epochs = 1000 #200,500,1000
dim_au_in = 11833
dim_au_out = 512 #8, 16, 32, 64, 128, 256,512
dim_dnn_in = dim_au_out
dim_dnn_out=1

# Import data

In [4]:
data_r=pd.read_csv('gdsc_gene_cell.csv')
label_r=pd.read_csv('gdsc_drug.csv')

In [5]:
label_r=label_r.fillna(1)

# Your is gene-cell, mine is cell-gene

In [6]:
data_r.head()

,BxPC-3,KMOE-2,MFM-223,NUGC-3,OC-314,COLO-741,KARPAS-45,JAR,DU-4475,MONO-MAC-6,...,SF268,TK10,A2780,OVCAR-4.1,A375,KP-N-S19s,BC-3,Panc 08.13.1,EKVX.1,DMS-114.1
ENSG00000000003_at,10.898588,7.294736,5.116163,9.865415,9.853360,8.530792,8.176877,10.006412,5.477983,5.104350,...,8.338001,9.008451,9.239745,9.472819,7.968322,7.993171,5.322980,9.665734,9.364049,8.128921
ENSG00000000005_at,4.201139,4.336010,4.170541,4.069192,4.010374,4.017925,4.519250,4.278360,4.434947,4.403349,...,4.349856,4.121786,4.177905,4.227348,4.198914,4.359674,4.366423,4.166249,4.294723,4.467436
ENSG00000000419_at,11.556533,11.106639,11.614775,11.670086,11.057632,10.646966,10.878239,11.017995,10.723468,10.504204,...,11.245667,11.175300,11.485499,11.504745,11.485343,11.189191,9.702628,10.958903,10.247844,12.187419
ENSG00000000457_at,5.453730,6.005694,5.865642,6.224943,5.149241,5.121324,5.868385,5.735913,6.644952,5.664282,...,5.387504,5.961516,5.988397,5.699127,6.254533,6.575614,5.885715,5.281582,5.453070,5.657234
ENSG00000000460_at,5.579061,6.482126,6.111542,5.722082,5.086321,6.260783,6.737340,6.782542,5.710254,5.971068,...,5.814175,5.514232,6.760882,6.022538,6.368033,6.994632,5.069775,5.521107,5.123464,5.369075


In [7]:
data = data_r.T
label = label_r.T.iloc[:,10]
scaler = preprocessing.StandardScaler(with_mean=True, with_std=True)
data = scaler.fit_transform(data)

In [8]:
print(np.std(data))
print(np.mean(data))

0.9999999999999999
1.7364322541407683e-17


In [9]:
data.shape

(789, 11833)

In [10]:
label_r.shape

(139, 789)

# Split test train

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(data, label, test_size=0.2, random_state=42)

In [12]:
print(data.shape)
print(label.shape)
print(X_train.shape, Y_train.shape)
print(X_test.shape, Y_test.shape)

(789, 11833)
(789,)
(631, 11833) (631,)
(158, 11833) (158,)


In [13]:
print(X_train.max())
print(X_train.min())

23.789515962677616
-17.27771827263068


In [14]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# Assuming that we are on a CUDA machine, this should print a CUDA device:
print(device)
torch.cuda.set_device(device)

cuda:0


# Add all data to AE

In [15]:
trainData = torch.FloatTensor(X_train).to(device)
testData = torch.FloatTensor(X_test).to(device)
y = torch.FloatTensor(Y_train.values).to(device)
allData = torch.FloatTensor(data).to(device)

# construct TensorDataset
train_dataset = TensorDataset(trainData, trainData)
test_dataset = TensorDataset(testData, testData)
all_dataset = TensorDataset(allData, allData)

trainDataLoader1 = DataLoader(dataset=train_dataset, batch_size=200, shuffle=False)
trainDataLoaderall = DataLoader(dataset=all_dataset, batch_size=200, shuffle=False)

# The model

In [16]:
# Construct autoencoder model

class AE(nn.Module):
    def __init__(self,dim_au_in,dim_au_out):
        super(AE, self).__init__()
        self.dim = dim_au_in
        self.fc1 = nn.Linear(dim_au_in, 1024)

        self.fc2 = nn.Linear(1024, dim_au_out)
        self.fc3 = nn.Linear(dim_au_out, 1024)

        self.fc4 = nn.Linear(1024, dim_au_in)            
                    
    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        return self.fc2(h1)
       
    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        return torch.sigmoid(self.fc4(h3))

    def forward(self, x):
        z = self.encode(x.view(-1, self.dim))
        #return self.decode(z), x
        return self.decode(z), z
    

In [17]:
# Initialization

#autoencoder = AE(dim=train_dataset.shape[1]).to(device)
autoencoder = AE(dim_au_in = X_train.shape[1],dim_au_out=dim_au_out).to(device)
optimizer = optim.Adam(autoencoder.parameters(), lr=1e-3)
#loss1
loss_func = nn.SmoothL1Loss().to(device)
#loss2
#loss_func = nn.BCELoss()
loss_train = np.zeros((epochs, 1))

In [18]:
# Train autoencoder

for epoch in range(epochs):
    # 不需要label，所以用一个占位符"_"代替
    for batchidx, (x, _) in enumerate(trainDataLoaderall):
        x.requires_grad_(True)
        # encode and decode 
        decoded, encoded = autoencoder(x)
        # compute loss
        print(encoded.shape, decoded.shape)
        loss = loss_func(decoded, x)      
        # update
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
           
    loss_train[epoch,0] = loss.item()  
    print('Epoch: %04d, Training loss=%.8f' %
          (epoch+1, loss.item()))

torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0001, Training loss=0.39214054
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0002, Training loss=0.38960111
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0003, Training loss=0.38965294
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0004, Training loss=0.38957641
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Siz

torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0036, Training loss=0.37307850
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0037, Training loss=0.37248147
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0038, Training loss=0.37156221
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0039, Training loss=0.37104896
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Siz

torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0072, Training loss=0.33057588
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0073, Training loss=0.33114591
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0074, Training loss=0.33227617
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0075, Training loss=0.32884023
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Siz

torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0107, Training loss=0.30056167
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0108, Training loss=0.30023095
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0109, Training loss=0.30033717
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0110, Training loss=0.30004439
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Siz

torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0143, Training loss=0.29062465
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0144, Training loss=0.29048854
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0145, Training loss=0.29045653
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0146, Training loss=0.29042736
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Siz

...

torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0853, Training loss=0.25458139
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0854, Training loss=0.25456652
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0855, Training loss=0.25451139
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0856, Training loss=0.25447148
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Siz

torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0888, Training loss=0.25390592
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0889, Training loss=0.25382447
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0890, Training loss=0.25381243
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0891, Training loss=0.25382203
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Siz

torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0924, Training loss=0.25348663
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0925, Training loss=0.25342596
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0926, Training loss=0.25338084
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0927, Training loss=0.25336912
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Siz

torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0959, Training loss=0.25303948
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0960, Training loss=0.25308424
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0961, Training loss=0.25307861
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0962, Training loss=0.25306484
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Siz

torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0995, Training loss=0.25279856
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0996, Training loss=0.25279507
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0997, Training loss=0.25280178
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([189, 512]) torch.Size([189, 11833])
Epoch: 0998, Training loss=0.25284064
torch.Size([200, 512]) torch.Size([200, 11833])
torch.Size([200, 512]) torch.Siz

In [19]:
# extract features
_, encodedTrainData = autoencoder(trainData)
featureTensor = encodedTrainData.double()
feature = featureTensor.detach().cpu().numpy()

In [20]:
print(feature.shape)

(631, 512)


In [21]:
feature

array([[ -5.61200714,  -7.60230827, -37.00702667, ...,  31.06952286,
         -0.63069469,   8.28977871],
       [ 23.75664139,  -8.77522945, -31.87708282, ...,  25.43472099,
        -16.0959549 ,  -2.49275184],
       [ 16.69183731,  19.71079254, -25.21900558, ...,  15.54164982,
        -21.05350304,   9.05363274],
       ...,
       [ 14.95630264,   0.49559677, -22.35454369, ...,  31.41143799,
         -5.73392582,  12.00165081],
       [  7.97794485,  -0.35433251, -10.12464523, ...,  11.52624893,
        -20.78687477,   4.81750154],
       [ 26.66612816,  -4.71366119,  -8.48579884, ...,  -4.50175381,
          2.71059585,   1.98805594]])

In [22]:
from sklearn import linear_model
clf = linear_model.Lasso(alpha=0.1)
clf.fit(feature, Y_train.values)

Lasso(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

In [23]:
_,testFeature = autoencoder(testData)
lasso = clf.predict(testFeature.detach().cpu().numpy())

In [24]:
r2_score(lasso,Y_test)

-2.4679801205952607

In [25]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components=2, verbose=1, perplexity=40, n_iter=300)
tsne_results = tsne.fit_transform(feature)

[t-SNE] Computing 121 nearest neighbors...
[t-SNE] Indexed 631 samples in 0.008s...
[t-SNE] Computed neighbors for 631 samples in 0.264s...
[t-SNE] Computed conditional probabilities for sample 631 / 631
[t-SNE] Mean sigma: 114.235404
[t-SNE] KL divergence after 250 iterations with early exaggeration: 73.638641
[t-SNE] KL divergence after 300 iterations: 1.600586


In [26]:
tsne_results

array([[  3.0418184 ,  -5.3938417 ],
       [ -9.691797  ,   1.2817073 ],
       [  2.2566214 , -13.448644  ],
       ...,
       [  3.6552374 ,   0.03919946],
       [  1.2460555 ,   5.335146  ],
       [ 10.667838  ,   6.5417023 ]], dtype=float32)

In [27]:
import matplotlib.pyplot as plt
import seaborn as sns

plt.scatter(tsne_results[:, 0], tsne_results[:, 1], label="label")
plt.legend()
plt.show()

<Figure size 640x480 with 1 Axes>

In [28]:
# Construct predictor

class DNN(nn.Module):
    def __init__(self,dim_dnn_in,dim_dnn_out):
        super(DNN, self).__init__()
        self.dim = dim_dnn_in
        #self.fc1 = nn.Linear(dim, 64)
        #self.fc2 = nn.Linear(64, 32)
        #self.fc3 = nn.Linear(32, 16)
        #self.fc4 = nn.Linear(16, num)
        self.fc1 = nn.Linear(dim_dnn_in, 128)
        self.fc2 = nn.Linear(128, 128)
        #self.fc3 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(128, dim_dnn_out)
        #self.fc4 = nn.Linear(64, num)
        #sigmoid=0/1
        self.sigmoid = torch.nn.Sigmoid()

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        #x = F.relu(self.fc3(x))
        y_hat=self.sigmoid(self.fc3(x))    
        #return self.decode(z), x
        return y_hat

In [29]:
EPOCH = 500

In [30]:
# Load data
# data type conversion
B_feature = torch.FloatTensor(feature).to(device)
y = torch.FloatTensor(Y_train.values).to(device)
# construct TensorDataset
b_data = TensorDataset(B_feature, y)
trainDataLoader2 = DataLoader(dataset=b_data, batch_size=200, shuffle=False)

In [31]:
b_data.tensors

(tensor([[ -5.6120,  -7.6023, -37.0070,  ...,  31.0695,  -0.6307,   8.2898],
         [ 23.7566,  -8.7752, -31.8771,  ...,  25.4347, -16.0960,  -2.4928],
         [ 16.6918,  19.7108, -25.2190,  ...,  15.5416, -21.0535,   9.0536],
         ...,
         [ 14.9563,   0.4956, -22.3545,  ...,  31.4114,  -5.7339,  12.0017],
         [  7.9779,  -0.3543, -10.1246,  ...,  11.5262, -20.7869,   4.8175],
         [ 26.6661,  -4.7137,  -8.4858,  ...,  -4.5018,   2.7106,   1.9881]],
        device='cuda:0'),
 tensor([4.1360e-03, 2.4651e-01, 1.0000e+00, 1.6831e-01, 2.4553e-01, 4.5574e-02,
         5.6766e-01, 6.5160e-03, 5.1101e-02, 1.0000e+00, 1.0000e+00, 1.3382e-01,
         8.5730e-03, 2.0783e-02, 7.1891e-02, 2.6463e-02, 2.0267e-01, 1.6792e-01,
         2.2742e-01, 5.1173e-01, 5.5870e-02, 9.3305e-02, 3.9400e-03, 9.9018e-02,
         1.0000e+00, 7.3892e-02, 1.1134e-02, 8.2544e-02, 1.1754e-02, 1.4302e-01,
         1.0000e+00, 1.0000e+00, 2.4001e-02, 3.9460e-03, 7.5210e-03, 4.9375e-01,
         7.

In [32]:
b_data

In [33]:
dim_dnn_in

512

In [34]:
# Initialization DNN model

predictor = DNN(dim_dnn_in, dim_dnn_out).to(device)
optimizer = optim.Adam(predictor.parameters(), lr=1e-3,betas=(0.9,0.99))
#loss1-softmax
#loss_func = nn.MSELoss()
#loss2-sigmoid
loss_func = nn.BCELoss().to(device)
#loss3-sigmoid
#loss_func = nn.CrossEntropyLoss()

#criterion = torch.nn.MSELoss(size_average=True)
#criterion = torch.nn.BCELoss(size_average=True) # Defined loss function
#optimizer = optim.Adm(model.parameters(), lr=0.01) # Defined optimizer
loss_train = np.zeros((epochs, 1))

In [35]:
# train model
for epoch in range(EPOCH):
    print('Epoch: ',epoch)
    for step,(batch_x,batch_y) in enumerate(trainDataLoader2):
        b_x = Variable(batch_x)
        b_y = Variable(batch_y)
        # predict label
        output = predictor(b_x)
        # b_y=F.sigmoid(b_y) 
        
        #print
        #print(output)
        #print(b_y)
        # compute loss
        loss = loss_func(output,b_y)
        #loss = criterion(output, b_y)
        
        # update
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    loss_train[epoch,0] = loss.item()  
    print('Epoch: %04d, Training loss=%.8f' %
          (epoch+1, loss.item())) 


Epoch:  0


D:\Anaconda3\lib\site-packages\torch\nn\modules\loss.py:529: UserWarning: Using a target size (torch.Size([200])) that is different to the input size (torch.Size([200, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)
D:\Anaconda3\lib\site-packages\torch\nn\modules\loss.py:529: UserWarning: Using a target size (torch.Size([200])) that is different to the input size (torch.Size([200, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)
D:\Anaconda3\lib\site-packages\torch\nn\modules\loss.py:529: UserWarning: Using a target size (torch.Size([200])) that is different to the input size (torch.Size([200, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)
D:\Anaconda3\lib\site-packages\torch\nn\

Epoch: 0001, Training loss=0.71679217
Epoch:  1


D:\Anaconda3\lib\site-packages\torch\nn\modules\loss.py:529: UserWarning: Using a target size (torch.Size([200])) that is different to the input size (torch.Size([200, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)
D:\Anaconda3\lib\site-packages\torch\nn\modules\loss.py:529: UserWarning: Using a target size (torch.Size([200])) that is different to the input size (torch.Size([200, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)
D:\Anaconda3\lib\site-packages\torch\nn\modules\loss.py:529: UserWarning: Using a target size (torch.Size([200])) that is different to the input size (torch.Size([200, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)
D:\Anaconda3\lib\site-packages\torch\nn\

Epoch: 0002, Training loss=0.50055659
Epoch:  2


...

Epoch: 0498, Training loss=0.49226469
Epoch:  498


D:\Anaconda3\lib\site-packages\torch\nn\modules\loss.py:529: UserWarning: Using a target size (torch.Size([200])) that is different to the input size (torch.Size([200, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)
D:\Anaconda3\lib\site-packages\torch\nn\modules\loss.py:529: UserWarning: Using a target size (torch.Size([200])) that is different to the input size (torch.Size([200, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)
D:\Anaconda3\lib\site-packages\torch\nn\modules\loss.py:529: UserWarning: Using a target size (torch.Size([200])) that is different to the input size (torch.Size([200, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)
D:\Anaconda3\lib\site-packages\torch\nn\

Epoch: 0499, Training loss=0.49634716
Epoch:  499


D:\Anaconda3\lib\site-packages\torch\nn\modules\loss.py:529: UserWarning: Using a target size (torch.Size([200])) that is different to the input size (torch.Size([200, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)
D:\Anaconda3\lib\site-packages\torch\nn\modules\loss.py:529: UserWarning: Using a target size (torch.Size([200])) that is different to the input size (torch.Size([200, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)
D:\Anaconda3\lib\site-packages\torch\nn\modules\loss.py:529: UserWarning: Using a target size (torch.Size([200])) that is different to the input size (torch.Size([200, 1])) is deprecated. Please ensure they have the same size.
  return F.binary_cross_entropy(input, target, weight=self.weight, reduction=self.reduction)
D:\Anaconda3\lib\site-packages\torch\nn\

Epoch: 0500, Training loss=0.49209717


In [36]:
# Get tesing feature
_,testFeature = autoencoder(testData)

In [37]:
testpredict = predictor(testFeature)

In [38]:
testpredict

tensor([[0.2873],
        [0.2097],
        [0.2486],
        [0.1554],
        [0.2231],
        [0.2878],
        [0.2032],
        [0.2341],
        [0.1928],
        [0.2943],
        [0.1636],
        [0.1928],
        [0.1760],
        [0.2328],
        [0.2951],
        [0.2709],
        [0.1782],
        [0.2913],
        [0.2764],
        [0.2694],
        [0.2350],
        [0.2420],
        [0.1954],
        [0.2858],
        [0.2108],
        [0.3133],
        [0.1845],
        [0.3090],
        [0.1512],
        [0.2473],
        [0.1102],
        [0.1706],
        [0.2516],
        [0.2666],
        [0.2159],
        [0.2652],
        [0.3158],
        [0.3271],
        [0.1889],
        [0.2606],
        [0.2341],
        [0.2294],
        [0.2024],
        [0.3140],
        [0.2582],
        [0.2413],
        [0.2171],
        [0.3022],
        [0.2860],
        [0.2200],
        [0.1955],
        [0.2689],
        [0.2198],
        [0.3066],
        [0.2727],
        [0

In [39]:
from sklearn.metrics import mean_squared_error

In [40]:
r2_score(testpredict.detach().cpu().numpy(),Y_test)

-54.36340798383833

In [41]:
mean_squared_error(testpredict.detach().cpu().numpy(),Y_test)

0.1298168323356889